In [ ]:
#Mon ini (SON)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/M1_S2/RITAL/projet2
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install python-terrier
import pandas as pd

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [2]:
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

def prepretraiter(query, ponctuation=True, stem=True, lemme=False, stopword=True, pos_tagging = False):
    """
    Préprétraite une query
    """
    #wn_lemmatizer = nltk.stem.WordNetLemmatizer()
    stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    stopwords_en = stopwords.words('english')
    punc = string.punctuation+'\n\r\t'
    q_traite = []
    for q in query:
        if not pos_tagging:
          q = q[0].lower()
          if ponctuation:
            q = q.translate(str.maketrans(punc, ' '*len(punc)))
          if stopword:
            q = word_tokenize(q)
            q = ' '.join([w for w in q if w not in stopwords_en])
          #q = ' '.join(map(lambda x: wn_lemmatizer.lemmatize(x), word_tokenize(q)))
          if lemme:
            q = ' '.join(map(lambda x: wordnet_lemmatizer.lemmatize(x), word_tokenize(q)))
          if stem:
            q = ' '.join(map(lambda x: stemmer.stem(x), word_tokenize(q)))
          q_traite.append(q)
        else:
          q = q[0]
          if ponctuation:
            q = q.translate(str.maketrans(punc, ' '*len(punc)))
          if stopword:
            q = word_tokenize(q)
            q = ' '.join([w for w in q if w.lower() not in stopwords_en])
          if lemme:
            q = word_tokenize(q)
            q = ' '.join(map(lambda x: wordnet_lemmatizer.lemmatize(x), word_tokenize(q)))
          q = word_tokenize(q)
          q = nltk.pos_tag(q)
          q = ' '.join([(w1.lower()+w2) if w2 != 'NNP' else (w1+w2) for w1, w2 in q]) # On lower pas les noms propres
          q_traite.append(q)
    return q_traite

def applique_pretraite_passage (df, ponctuation=True, stem=True,lemme=False, stopword=True, pos_tagging = False) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['text']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword, pos_tagging = pos_tagging)
    df['text']=liste_query
    return df

def applique_pretraite_query (df, ponctuation=True, stem=True,lemme=False, stopword=True, pos_tagging = False) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['query']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword, pos_tagging = pos_tagging)
    df['query']=liste_query
    df = df.astype({"qid": str, "query": object})
    
    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Préparation de données

In [3]:
# chargement du json en dataframe
with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
data={"DocumentID" : [int(k0) for k0,v0 in data_row.items() for k1,v1 in v0.items()],
    "docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
      "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
docs_df = pd.DataFrame(data)

In [4]:
ponctuation=True
stem=False
lemme=False
stopword=True
postag = True
docs_df = applique_pretraite_passage (docs_df, ponctuation=ponctuation, stem=stem,lemme=lemme, stopword=stopword,pos_tagging = postag)
docs_df

,DocumentID,docno,text
0,344,"344,1",TownspeopleNNP livedVBD charteredJJ townsNNS b...
1,344,"344,0",municipalJJ corporationNN legalJJ termNN local...
2,344,"344,3",standardJJ variesNNS stateNN stateNN according...
3,344,"344,2",municipalJJ corporationsNNS mustMD alsoRB abid...
4,344,"344,5",panchayatiNN rajNN systemNN interactsVBZ direc...
...,...,...,...
50607,479,"479,18",dialectsNNS alsoRB dividedVBD twoCD mainJJ chr...
50608,479,"479,31",officialJJ languageNN MoscowNNP NovgorodNNP la...
50609,479,"479,30",soonRB followedVBD adoptionJJ ChristianityNNP ...
50610,479,"479,33",knamNNS vaVBP koskJJ zNN stuNN tcitJJ politica...


## Query

In [5]:
df_train = pd.read_csv("WikiPassageQA/train.tsv",sep='\t')
df_test = pd.read_csv("WikiPassageQA/test.tsv",sep='\t')
df_dev = pd.read_csv("WikiPassageQA/dev.tsv",sep='\t')
#Question None
df_train = df_train[df_train['Question'] != '{}']
df_test = df_test[df_test['Question'] != '{}']
df_dev = df_dev[df_dev['Question'] != '{}']

queries_train=df_train[["QID", "Question"]]
queries_train.columns = ['qid', 'query']
queries_test=df_test[["QID", "Question"]]
queries_test.columns = ['qid', 'query']
queries_dev=df_dev[["QID", "Question"]]
queries_dev.columns = ['qid', 'query']
df_query = pd.concat([df_train, df_test, df_dev])
df_complete = df_query.merge(docs_df, on =["DocumentID"])
df_complete

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",claimedVBN majorJJ creditNN electionNN Fernand...
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",EvangelicalismNNP majorJJ forceNN AnglicanNNP ...
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",richJJ poorJJ remainedVBD traditionalJJ Cathol...
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49",evangelicalJJ AllianceNNP formedVBD 1846CD fir...
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24",1737CD WhitefieldNNP becomeVBD nationalJJ cele...
...,...,...,...,...,...,...,...
244014,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,72",budgetaryJJ ruleNN spendVBP 4CD fundNN yearNN ...
244015,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,71",NorwayNNP fifthRB largestJJS oilNN exporterNN ...
244016,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,70",2011CD 28CD stateNN revenuesNNS generatedVBD p...
244017,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,79",twoCD prominentJJ E6NNP goingVBG northJJ south...


# Jugement de pertinence

In [6]:
relevant = []
for row in df_complete.iterrows():
  passage = row[1]['RelevantPassages'].split(",")
  doc_passage = row[1]['docno'].split(",")[1] # doc,pass -> [1]
  if doc_passage in passage:
    relevant.append(1)
  else:
    relevant.append(0)

df_complete["label"] = relevant
qrel = df_complete[["QID", "docno","label"]]
qrel.columns = ['qid', 'docno', 'label']
qrel = qrel.astype({"qid": str, "docno": object, "label":object}) #qid en string pour pt.Experiment
qrel

,qid,docno,label
0,3086,"672,42",0
1,3086,"672,48",0
2,3086,"672,43",0
3,3086,"672,49",0
4,3086,"672,24",0
...,...,...,...
244014,2570,"553,72",0
244015,2570,"553,71",0
244016,2570,"553,70",0
244017,2570,"553,79",0


In [7]:
#avec les configurations identiques pour les textes du passage
queries_train=applique_pretraite_query(queries_train,ponctuation=True, stem=stem,lemme=lemme, stopword=stopword,pos_tagging = postag)
queries_test=applique_pretraite_query(queries_test,ponctuation=True, stem=stem,lemme=lemme, stopword=stopword,pos_tagging = postag)
queries_dev=applique_pretraite_query(queries_dev,ponctuation=True, stem=stem,lemme=lemme, stopword=stopword,pos_tagging = postag)

queries_test

<ipython-input-2-82fa2d757c70>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-2-82fa2d757c70>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-2-82fa2d757c70>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,qid,query
0,449,IraqNNP roleNN politicalJJ unstabilizationNN
1,1140,warmestJJS yearsNNS globalJJ warmingNN
2,782,OlympicNNP gamesNNS broadcastedVBD
3,1421,electionNN turnoutIN 2008CD
4,3738,secularismNN misinterpretedVBD AmericaNNP
...,...,...
411,2522,churchNN meanNN FallenNNP natureNN
412,2851,historyNN eurasiaNN
413,3705,nicknamesNNS MissouriNNP
414,799,endoglossicJJ exoglossicJJ languagesNNS coexis...


# BERT

In [8]:
# Paquets pour BERT
!pip install faiss-gpu
!apt install --upgrade libomp-dev

!pip install --upgrade transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp-10-dev libomp5-10
Suggested packages:
  libomp-10-doc
The following NEW packages will be installed:
  libomp-10-dev libomp-dev libomp5-10
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 351 kB of archives.
After this operation, 2,281 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp5-10 amd64 1:10.0.0-4ubuntu1 [300 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp-10-dev amd64 1:10.0.0-4ubuntu1 [47.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libomp-dev amd64 1:10.0-50~exp1 [2,824 B]
Fetched 351 kB in 2s (184 kB/s)
Selecting pr

In [9]:
# plugin

!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/terrierteam/pyterrier_colbert.git to /tmp/pip-req-build-iu8ux0iz
  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_colbert.git /tmp/pip-req-build-iu8ux0iz
  Resolved https://github.com/terrierteam/pyterrier_colbert.git to commit b551d4dc26fcfac5d68cf41a9738fedf012caee1
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to /tmp/pip-install-wsszx1fs/colbert_d542bdc7bb2f487fbb0a6373096a650a
  Running command git clone --filter=blob:none --quiet https://github.com/cmacdonald/ColBERT.git /tmp/pip-install-wsszx1fs/colbert_d542bdc7bb2f487fbb0a6373096a650a
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.
  Resolved https://github.com/cmacdonald/ColBERT.

In [10]:
!rm -rf /content/colbert_index

import pyterrier_colbert.indexing
# BERT checkpoint
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"
colbert_indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint=checkpoint, 
                                                            index_root="/content",
                                                            index_name="colbert_index",
                                                            chunksize=3)

In [11]:
from typing import Union, Tuple, Iterator, Dict, Any, List

class GeneratorLen(object):
    def __init__(self, gen, length):
        self.gen = gen
        self.length = length

    def __len__(self): 
        return self.length

    def __iter__(self):
        return self.gen

def custom_get_corpus_iter(df: pd.DataFrame) -> Iterator[Dict[str,Any]]:
  len_total = df.shape[0]
  def gen():
    for _, row in df.iterrows():
      yield {"docno": row["docno"], "text": row["text"]}

  return GeneratorLen(gen(), len_total)

In [12]:
colbert_indexer.index(custom_get_corpus_iter(docs_df))

[May 20, 16:33:56] [0] 		 #> Local args.bsize = 128
[May 20, 16:33:56] [0] 		 #> args.index_root = /content
[May 20, 16:33:56] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 20, 16:33:59] #> Loading model checkpoint.
[May 20, 16:33:59] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


Downloading: "http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip" to /root/.cache/torch/hub/checkpoints/colbert_model_checkpoint.zip
100%|██████████| 1.11G/1.11G [01:29<00:00, 13.2MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 20, 16:35:41] #> checkpoint['epoch'] = 0
[May 20, 16:35:41] #> checkpoint['batch'] = 44500






[May 20, 16:35:44] #> Note: Output directory /content already exists




[May 20, 16:35:44] #> Creating directory /content/colbert_index 


[May 20, 16:37:51] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 23.9k (overall),  24.3k (this encoding),  78140.7M (this saving)
[May 20, 16:37:54] [0] 		 [NOTE] Done with local share.
[May 20, 16:37:54] [0] 		 #> Joining saver thread.
[May 20, 16:37:55] [0] 		 #> Saved batch #0 to /content/colbert_index/0.pt 		 Saving Throughput = 704.0k passages per minute.

#> num_embeddings = 8721535
[May 20, 16:37:55] #> Starting..
[May 20, 16:37:55] #> Processing slice #1 of 1 (range 0..1).
[May 20, 16:37:55] #> Will write to /content/colbert_index/ivfpq.100.faiss.
[May 20, 16:37:55] #> Loading /content/colbert_index/0.sample ...
#> Sample has shape (436076, 128)
[May 20, 16:37:55] Preparing resources for 1 GPUs.
[May 20, 16:37:55] #> Training with the vectors...
[May 20, 16:37:55] #> Training now (using 1 GPUs)...
952.4226641654968

In [13]:
#LEMME_POSTAG
colbert_retriever = colbert_indexer.ranking_factory().end_to_end()

pt.Experiment(
    [colbert_retriever], 
    queries_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names=['ColBERT'],
)

[May 20, 16:54:05] #> Loading the FAISS index from /content/colbert_index/ivfpq.100.faiss ..
[May 20, 16:54:05] #> Building the emb2pid mapping..
[May 20, 16:54:05] len(self.emb2pid) = 8721535
Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,ColBERT,0.311549,0.387904,0.110577,0.067788,0.452207,0.384095,0.451042,0.519551


In [ ]:
#LEMME
colbert_retriever = colbert_indexer.ranking_factory().end_to_end()

pt.Experiment(
    [colbert_retriever], 
    queries_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names=['ColBERT'],
)

[May 18, 17:40:01] #> Loading the FAISS index from /content/colbert_index/ivfpq.100.faiss ..
[May 18, 17:40:01] #> Building the emb2pid mapping..
[May 18, 17:40:01] len(self.emb2pid) = 4861412
Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,ColBERT,0.467077,0.555638,0.163942,0.09976,0.599076,0.552244,0.645873,0.711859
